In [1]:
# Set root directory
import os

ROOT_DIR = "D:\Coding\CZ4010\Applied-Cryptography-Project"
os.chdir(ROOT_DIR)

In [2]:
from utils import ascii_to_binary_list, binary_list_to_ascii
from LWE_PKC import LWE_Encrypt, LWE_Decrypt
from math import comb

### 1. Testing out the Cryptosystem


In [5]:
bit_message = [1]

In [6]:
# Weak Parameters
n = 3
q = 17
max_error = 1
S = 2*max_error + 1   # Cardinality of the error set
m = 2*comb(n+S, S)    # The number of eqations must be >> (n+S)C(s)

In [7]:
# Initialize PKC with parameters
lwe_d = LWE_Decrypt(n=n, q=q, max_error=max_error, list_size=m)

# Get public keys
A_list, T_list, q, max_error = lwe_d.get_public_keys()
# print(A_list)
# print(T_list)
print(f"Randomly Initalized Secret: {lwe_d.secret}")

Randomly Initalized Secret: [9, 2, 2]


In [8]:
# Encrypt Message
lwe_e = LWE_Encrypt(A_list, T_list, q, max_error)
A_new, T_send = lwe_e.encrypt_message(bit_message)
print(f"A_new = {A_new}\nT_send = {T_send}")

A_new = [array([11, 15, 11], dtype=int32)]
T_send = [8]


In [9]:
# Decrypt Message
decrypted_binary_message = lwe_d.decrypt_message(A_new, T_send)
decrypted_message = binary_list_to_ascii(decrypted_binary_message)
print(f"Decrypted Message = {decrypted_binary_message}")

Decrypted Message = [1]


### Arora-Ge Attack

An algebraic attack when the max_error parameter is small.

`https://eprint.iacr.org/2014/1018.pdf`


In [272]:
# Weak Parameters
n = 5
q = 103
max_error = 3
E = 2*max_error + 1   # Cardinality of the error set

# Set the number of samples wisely!
# m = int(2 * comb(n+E, E))   # The number of eqations must be >> (n+S)C(s)
# m = int(2 * n ** (E))
m = 1000

In [273]:
# Initialize PKC with parameters
lwe_d = LWE_Decrypt(n=n, q=q, max_error=max_error, list_size=m)

# The secret is randomly initalized in the class
A_list, b_list, q, max_error = lwe_d.get_public_keys()
print(f"Randomly Initalized Secret: {lwe_d.secret}")

Randomly Initalized Secret: [82, 21, 39, 30, 88]


We are going to recover the secret `s` from `A_list` and `b_list` given the following weak parameters,

1. The error is truncated and takes values in the set $\{-T, ... , -1, 0, 1, ... , T\} \implies |E| = 2T + 1$
   <br>
   <br>
2. The number of equations in `A_list`, $m >> {n + |E| \choose |E|}$


In [274]:
print(f"Cardinality of error set: {E}")
print(f"Number of equations: {len(A_list)} >> {comb(n+E, E)}")

Cardinality of error set: 7
Number of equations: 1000 >> 792


In [275]:
from itertools import product
from collections import defaultdict
from sympy import symbols, Matrix, GF
from sympy.polys.matrices import DomainMatrix

In [276]:
# Define the secret vector
secret_vector = symbols(f'x1:{n+1}')
secret_vector

(x1, x2, x3, x4, x5)

In [277]:
polynomials_over_Zq = []
error_set = [i for i in range(-max_error, max_error+1)] 

# Begin constructing the polynomials for each LWE instance <A, b>
for A, b in zip(A_list, b_list):
    # Initalize the polynomial term to the identity polynomial of the finite field
    polynomial_over_Zq = GF(q)[secret_vector](1)

    for e in error_set:
        # Multiply each variable by its corresponding weight
        weighted_polynomial = sum(w * var for w, var in zip(A, secret_vector))
        # print(weighted_polynomial)

        # Construct the weighted polynomial (this is the AT*s term in the equation)
        weighted_secret_polynomial_over_Zq = GF(q)[secret_vector](weighted_polynomial)
        # print(weighted_secret_polynomial_over_Zq)

        # Complete the term (b - AT*s - e)
        term = b - weighted_secret_polynomial_over_Zq - e
        # print(term)

        # Accumulate the product of each term over Zq
        polynomial_over_Zq = polynomial_over_Zq * term
        # print(polynomial_over_Zq)
    
    # Collect the polynomials
    polynomials_over_Zq.append(polynomial_over_Zq)

# Sanity check
assert len(polynomials_over_Zq) == len(A_list)

In [278]:
def generate_tuples(n, d):
    # Use itertools.product to generate all tuples
    tuples = list(product(range(d + 1), repeat=n))
    
    # Remove tuples that have a degree > d
    tuples_pruned = [x for x in tuples if sum(x) <= d]

    return tuples_pruned

In [279]:
coefficients_dicts = []
tuples = generate_tuples(n, E)

for polynomial in polynomials_over_Zq:
    coefficients_dict = defaultdict(int, {key: 0 for key in tuples})

    for term_key, coeff in polynomial.terms():
        coefficients_dict[term_key] = int(coeff) # NOTE: Converting to negative int!!!

    coefficients_dicts.append(coefficients_dict)

In [280]:
# # Checking an example
# coefficients_dicts[0]

In [281]:
row_order = []

# Let's keep the degree 1 terms up front for convenince
for i in range(n):
    term = [0]*n
    term[i] = 1
    row_order.append(tuple(term))

# We want the secret at the start and the constant term at the end
seen = set(row_order)
constant_term_key = tuple([0]*n)
candidates = generate_tuples(n, E)

for x in candidates:
    if x not in seen and x != constant_term_key:
        row_order.append(x)

# Ensure the constant term is at the end
row_order.append(constant_term_key)

# Sanity check
assert len(row_order) == len(generate_tuples(n, E))

In [282]:
coefficient_matrix = []
rhs = []

for coeff_dict in coefficients_dicts:
    row = []

    for key in row_order[:-1]:
        row.append(coeff_dict[key])
    
    # Append row
    coefficient_matrix.append(row)

    # Append the rhs
    negative_constant_term = -coeff_dict[row_order[-1]]
    rhs.append(negative_constant_term)

In [283]:
# Solving linear system using DomainMatrix
m = Matrix(coefficient_matrix)
b = Matrix(rhs)

# Convert matrices to finite field of order q (q is prime):
K = GF(q, symmetric=False)
dm = DomainMatrix.from_Matrix(m).convert_to(K)
bm = DomainMatrix.from_Matrix(b).convert_to(K)

# Print shape of system of equations
print(dm.shape)
print(bm.shape)

# Solve and convert back to an ordinary Matrix:
solution_vector = dm.lu_solve(bm).to_Matrix()
print(solution_vector)

(1000, 791)
(1000, 1)
Matrix([[82], [21], [39], [30], [88], [19], [24], [52], [44], [61], [12], [65], [55], [102], [15], [84], [79], [76], [96], [2], [73], [38], [48], [14], [99], [60], [27], [7], [8], [86], [49], [89], [34], [5], [28], [93], [47], [9], [33], [20], [9], [71], [68], [10], [37], [63], [85], [64], [70], [83], [80], [36], [78], [66], [40], [31], [50], [74], [23], [3], [58], [57], [90], [92], [22], [79], [51], [59], [42], [91], [77], [1], [88], [19], [24], [52], [30], [65], [55], [102], [76], [96], [2], [14], [99], [8], [94], [32], [35], [93], [47], [39], [33], [20], [9], [37], [63], [85], [80], [36], [31], [61], [12], [26], [22], [79], [51], [59], [1], [88], [30], [10], [56], [87], [94], [32], [39], [81], [21], [61], [69], [97], [90], [92], [62], [100], [45], [12], [26], [22], [82], [6], [13], [51], [59], [42], [91], [77], [88], [19], [24], [52], [65], [55], [102], [96], [2], [99], [98], [75], [8], [86], [49], [89], [56], [87], [34], [5], [28], [32], [35], [93], [47], [33]

In [284]:
print(f"Randomly Initalized Secret:\t\t{lwe_d.secret}")
print(f"Secret obtained from Arora-Ge Attack:\t{solution_vector[:n]}")

Randomly Initalized Secret:		[82, 21, 39, 30, 88]
Secret obtained from Arora-Ge Attack:	[82, 21, 39, 30, 88]


Works pretty well for `n <= 5` E can be set to even 10 provided m is set accordingly.


### Efficient Implementation using Groebner Basis

Does not work very well, better implement in Sage.


In [285]:
# from sympy import symbols, Poly, GF, groebner

# def arora_ge_attack(q, A, b, E, S=None):
#     """
#     Recovers the secret key s from the LWE samples A and b.
#     More information: "The Learning with Errors Problem: Algorithms" (Section 1)
#     :param q: the modulus
#     :param A: the matrix A, represented as a list of lists
#     :param b: the vector b, represented as a list
#     :param E: the possible error values
#     :param S: the possible values of the entries in s (default: None)
#     :return: a list representing the secret key s
#     """
#     m = len(A)
#     n = len(A[0])

#     x = symbols(tuple(f"x{i}" for i in range(n)))
#     gf = GF(q)
#     pr = gf[x]
#     gens = pr.symbols

#     f = []
#     for i in range(m):
#         p = 1
#         for e in E:
#             p *= (b[i] - sum(A[i][j] * gens[j] for j in range(n)) - e)
#         f.append(p)

#     if S is not None:
#         # Use information about the possible values for s to add more polynomials.
#         for j in range(n):
#             p = 1
#             for s in S:
#                 p *= (gens[j] - s)
#             f.append(p)

#     ideal = [Poly(poly, gens) for poly in f]
#     basis = groebner(ideal, gens, order='lex')

#     s = []
#     for poly in basis:
#         #assert poly.variables() == 1 and poly.degree() == 1
#         s.append(int(-poly.coeffs()[0]))

#     return s

In [286]:
# error_values = [i for i in range(-max_error, max_error+1)] 
# x = arora_ge_attack(q=q, A=A_list, b=b_list, E=error_values)